## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.3190,84.52,73,1,3.00,light rain
1,1,Hithadhoo,MV,-0.6000,73.0833,83.41,73,100,9.19,light rain
2,2,Togur,RU,58.3631,82.8265,52.34,91,100,11.34,overcast clouds
3,3,Mitsamiouli,KM,-11.3847,43.2844,75.60,79,74,10.04,broken clouds
4,4,Puerto Cabezas,NI,14.0351,-83.3888,79.34,87,65,7.52,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Togur,RU,58.3631,82.8265,52.34,91,100,11.34,overcast clouds
7,7,Rikitea,PF,-23.1203,-134.9692,74.25,70,0,13.24,clear sky
11,11,Vestmannaeyjar,IS,63.4427,-20.2734,50.34,80,100,24.11,overcast clouds
14,14,Namibe,AO,-15.1961,12.1522,60.40,89,45,3.11,scattered clouds
17,17,Penzance,GB,50.1186,-5.5371,61.48,91,99,17.85,overcast clouds
18,18,Jalu,LY,29.0331,21.5482,73.29,45,0,9.46,clear sky
22,22,Astoria,US,46.1879,-123.8313,59.86,74,100,5.35,overcast clouds
23,23,Kodiak,US,57.7900,-152.4072,51.31,93,90,8.05,light rain
24,24,Semirom,IR,31.4167,51.5667,53.71,31,0,1.43,clear sky
26,26,Abu Samrah,SY,35.3029,37.1841,66.09,81,0,5.30,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                344
City                   344
Country                336
Lat                    344
Lng                    344
Max Temp               344
Humidity               344
Cloudiness             344
Wind Speed             344
Current Description    344
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')
clean_df.count()

City_ID                336
City                   336
Country                336
Lat                    336
Lng                    336
Max Temp               336
Humidity               336
Cloudiness             336
Wind Speed             336
Current Description    336
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Togur,RU,52.34,overcast clouds,58.3631,82.8265,
7,Rikitea,PF,74.25,clear sky,-23.1203,-134.9692,
11,Vestmannaeyjar,IS,50.34,overcast clouds,63.4427,-20.2734,
14,Namibe,AO,60.40,scattered clouds,-15.1961,12.1522,
17,Penzance,GB,61.48,overcast clouds,50.1186,-5.5371,
18,Jalu,LY,73.29,clear sky,29.0331,21.5482,
22,Astoria,US,59.86,overcast clouds,46.1879,-123.8313,
23,Kodiak,US,51.31,light rain,57.7900,-152.4072,
24,Semirom,IR,53.71,clear sky,31.4167,51.5667,
26,Abu Samrah,SY,66.09,clear sky,35.3029,37.1841,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [ ]:
hotel_df

In [13]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Rikitea,PF,74.25,clear sky,-23.1203,-134.9692,People ThankYou
11,Vestmannaeyjar,IS,50.34,overcast clouds,63.4427,-20.2734,Glamping & Camping
14,Namibe,AO,60.40,scattered clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
17,Penzance,GB,61.48,overcast clouds,50.1186,-5.5371,Artist Residence Penzance
18,Jalu,LY,73.29,clear sky,29.0331,21.5482,صاحبة الفخامة
...,...,...,...,...,...,...,...
661,Canoinhas,BR,52.81,overcast clouds,-26.1772,-50.3900,Santa Catarina Plaza Hotel
667,Oyem,GA,67.17,moderate rain,1.5995,11.5793,Commissariat de police
668,Sioux Lookout,CA,55.47,broken clouds,50.1001,-91.9170,Knobby's Fly-In Camps
673,Kargasok,RU,53.64,overcast clouds,59.0556,80.8572,Hotel Kargasok


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))